In [ ]:
# EU 
# For One Ye

Vorgehnsweise: 
Visualisierung in Pandas Darstellen 
Probe Dashboard mit plotly Dash erstellen 
Dashboar in Jupyter Book ein binden 
-> weiteren Grafiken umsetzen 

Ziele: 
Auswertung auf Basis EU 

Zeitlicher Verlauf gesamt
Abfragen:
    - Summe burned area pro Monat 
    - Fläche verbrannter Wälder insgesamt 
    - Liste besonders betroffener Länder 

Visualisierungen: 
    - Balken-Diagram: Summe burned area pro Monat 
    - Treemap: Verbrannte Fläche pro Vegetationstyp
    - Weltkarte mit betroffener Länder (gefüllt): Farbe Burned Area
    - DE-Karte: Bubles Größe (Burned area)


In [1]:
from pymongo import MongoClient
import pandas as pd
import plotly.express as px

In [ ]:
!"C:\Program Files\MongoDB\Server\6.0\bin\mongod.exe" --dbpath "C:\mongodb_data"

In [2]:
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
mydb = client["esa_fire"]
mycol = mydb["esa_fire_2001"]
print(mydb.name)

esa_fire


### Query 1: Flächen in Deutschland mit einer verbrannten Fläche größer als 1 km2

In [ ]:
# Querys Mongo DB
query2={
     '$and': [
            { 'country' : 'DE' },
            { 'burned_area'    : { '$gt': 1000000 }}
          ]
}
points =mycol.find(query2)
print("Anzahl von Städten mit Bevölkerungsdichte größer als 2000 in BaWü: ",mycol.count_documents(query2))

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[48.74388,9.10014],zoom_start=5)

In [ ]:
for citie in points:
    folium.Marker( location=[ citie['lat'], citie['lon'] ], fill_color='#43d9de', radius=8,tooltip=citie['date']).add_to(m)

In [ ]:
#m

### Query 2: Gruppierung der Flächen nach Datum

In [ ]:
list_date = list ()
agg_result= mycol.aggregate(
    [{
    "$group" : 
        {"_id" : "$date", 
         "burned_area" : {"$sum" : 1}
         }
    }])
for i in agg_result:
    list_date.append(i)
list_date

In [ ]:
import pandas as pd
df_date = pd.DataFrame(list_date)

In [ ]:
import plotly.express as px

fig = px.bar(df_date, x="_id", y="burned_area", title='Life expectancy in Canada')
fig.show()

In [ ]:
list_date = list ()
agg_result= mycol.aggregate(
    [{
    "$group" : 
        {"_id" : "$country", 
         "burned_area" : {"$sum" : 1}
         }
    }])
for i in agg_result:
    list_date.append(i)
list_date

In [ ]:
df_date = pd.DataFrame(list_date).sort_values(by=['burned_area'], ascending= False)
df_date

In [ ]:
fig = px.bar(df_date, x="_id", y="burned_area", title='Life expectancy in Canada')
fig.show()

## Verbrannte Fläche pro Vegetationstyp

In [ ]:
# Summe der Verbranten Fläche ->  pro Vegetatoionstyp
# Waterfall Diagramm

In [ ]:

list_veg = list ()
agg_result= mycol.aggregate([{ "$group":
  { "_id" : None,  
    "burned_area"  : { "$sum": "$burned_area" }, 
    "no_data_burned_area"  : { "$sum":"$no_data_burned_area" },
    "tree_cover_broadleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_broadleaved_deciduous_burned_area" },
    "tree_cover_broadleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_broadleaved_deciduous_burned_area" },
    "tree_cover_needleleaved_evergreen_burned_area"  : { "$sum":"$tree_cover_needleleaved_evergreen_burned_area" },
    "tree_cover_needleleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_needleleaved_deciduous_burned_area" },
    "tree_cover_mixed_leave_burned_area"  : { "$sum":"$tree_cover_mixed_leave_burned_area"},
    "cropland_burned_area"  : { "$sum":"$cropland_burned_area" },
    "mosaic_tree_grass_burned_area"  : { "$sum":"$mosaic_tree_grass_burned_area" },
    "flooded_area_burned_area"  : { "$sum":"$flooded_area_burned_area" },
    "other_vegetation_burned_area"  : { "$sum":"$other_vegetation_burned_area" },}
}])
for i in agg_result:
    list_veg.append(i)
list_veg

In [ ]:
df_veg = pd.DataFrame(list_veg)

In [ ]:
df_veg= df_veg.drop(columns="_id")

In [ ]:
df_veg_trans = df_veg.transpose()

In [ ]:
fig = px.bar(df_veg, title='Life expectancy in Canada')
fig.show()

In [ ]:
df_veg

In [ ]:
import plotly.graph_objects as go

fig1  = go.Figure()
hrz = ["burned_area", "no_data_burned_area","tree_cover_broadleaved_deciduous_burned_area",
       "tree_cover_needleleaved_evergreen_burned_area","tree_cover_needleleaved_deciduous_burned_area", "cropland_burned_area", "mosaic_tree_grass_burned_area", "flooded_area_burned_area", "other_vegetation_burned_area"
       "Final"]
vrt = [ df_veg["burned_area"][0], 
       -df_veg["no_data_burned_area"][0],
       -df_veg["tree_cover_broadleaved_deciduous_burned_area"][0],
       -df_veg["tree_cover_needleleaved_evergreen_burned_area"][0],
       -df_veg["tree_cover_needleleaved_deciduous_burned_area"][0],
       -df_veg["cropland_burned_area"][0],
       -df_veg["mosaic_tree_grass_burned_area"][0],
       -df_veg["flooded_area_burned_area"][0],
       -df_veg["other_vegetation_burned_area"][0],
       None]
fig1.add_trace(go.Waterfall(                 
                  x = hrz, y = vrt,
                  base = 0,
                  measure = [ "absolute","relative",    
                              "relative","relative",
                              "relative","total" ]                        
                ))                                 
fig1.show()



In [ ]:
# Abfrage EU Länder
# Umwandlung in Dataframe

## Dashboard Europa

In [3]:
df_countries = pd.read_csv('C:\dev\eda_esa_fire\doc\country_codes.csv', on_bad_lines='skip', delimiter=',')

In [4]:
europe = df_countries[df_countries['region'] == 'Europe']['alpha-2'].to_list()

In [5]:
list_europe = list(mycol.find({"country":{"$in":europe}}))
df_eu = pd.DataFrame(list_europe)

In [ ]:
df_eu

In [24]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data

app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    html.Label([
        "label",
    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "ES", "value": "ES"},
                     {"label": "GB", "value": "GB"},],
                 multi=False,
                 value=2015,
                 style={'width': "40%"}
                 ),
    dcc.Graph(id='graph'),
    ]),
])# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("slct_year", "value")]
)
def update_figure(slct_year):
    dff = df_eu.copy()
    dff = dff[dff["country"] == slct_year]
    return px.scatter(
        dff, x="lon", y="lat",
        render_mode="webgl", title="Tips"
    )# Run app and display result inline in the notebook
app.run_server(mode='inline')